## Train models - Detect abandoned villages in Nigeria

### Libraries

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [ ]:
# Check whether the kernel is backened with GPU
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
#gpu = len(tf.config.list_physical_devices('GPU'))>0
#print("GPU is", "available" if gpu else "NOT AVAILABLE")

# versions of tf and keras in p400
# Tensor Flow Version: 2.1.0
# Keras Version: 2.2.4-tf

Tensor Flow Version: 1.15.2
Keras Version: 2.2.4-tf

Python 3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0]
Pandas 1.1.3
Scikit-Learn 0.22.2.post1


In [ ]:
!pip install efficientnet

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from os import listdir
import random

import tensorflow as tf
print(tf.__version__)

from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense
from keras import optimizers
from keras.models import load_model
from efficientnet.keras import EfficientNetB0

from sklearn.model_selection import train_test_split

1.15.2


Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
path = "/content/drive/My Drive/PhD_Detect_Abandoned_Villages"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Helper functions

In [ ]:
def define_path():
    
    path_base = "C://Users//b1077901//PhD_Projects"
    path_array = os.path.join(path_base, "Array")
    path_model_base = os.path.join(path_base, "Models") 
    
    return path_array, path_model_base

# load image data
def load_x(data_version, img_type, buffer_len, data_type):
    
    x_name = "dataset" + "_" + str(data_version) + "_" + img_type + "_Buf" + str(buffer_len) + "_" + data_type + "_img.npy"
    x_path = os.path.join(path_array, x_name)
    x_img = np.load(x_path)
    print("img shape", x_img.shape)
    
    return x_img

def load_y(data_version, img_type, buffer_len, merge_type, data_type):
    
    y_name = "dataset" + "_" + str(data_version) + "_" + img_type + "_Buf" + str(buffer_len) + "_" + data_type + "_"  + merge_type + "_lab.npy"
    y_path = os.path.join(path_array, y_name)
    y_lab = np.load(y_path)
    print(merge_type)
    print("lab shape", y_lab.shape)

    return y_lab

# load train image data
def load_image_label_data(data_type, img_type_list, buffer_list, merge_type_list):

    data_version = 0
    img_list = []
    lab_partfull_list = []
    lab_partdest_list = []

    for i in range(len(img_type_list)):

        img_type = img_type_list[i]
        print("img_type", img_type)
        print("")

        for j in range(len(buffer_list)):

            buffer_len = buffer_list[j]
            print("Buffer", buffer_len)

            data_version += 1

            x_img = load_x(data_version, img_type, buffer_len, data_type)
            img_list.append(x_img)

            merge_type = merge_type_list[0]
            y_lab = load_y(data_version, img_type, buffer_len, merge_type, data_type)
            lab_partfull_list.append(y_lab)

            merge_type = merge_type_list[1]
            y_lab = load_y(data_version, img_type, buffer_len, merge_type, data_type)
            lab_partdest_list.append(y_lab)
            
            print("")
            
    return img_list, lab_partfull_list, lab_partdest_list

def get_image_label_dict(img_list, img_type_list, buffer_list, merge_type_list, lab_partfull_list, lab_partdest_list):
    
    version = 0
    img_dict = {}
    lab_partdest_dict = {}
    lab_partfull_dict = {}

    for i in range(len(img_type_list)):

        img_type = img_type_list[i]

        for j in range(len(buffer_list)):

            buffer_len = buffer_list[j]        
            dict_key = img_type+"_Buf"+str(buffer_len)
            
            img_dict[dict_key] = img_list[version]
            lab_partdest_dict[dict_key+"_"+merge_type_list[1]] = lab_partdest_list[version]
            lab_partfull_dict[dict_key+"_"+merge_type_list[0]] = lab_partfull_list[version]
            
            version += 1
            
    return img_dict, lab_partdest_dict, lab_partfull_dict

def eff_net_b0(buffer):
    
    # include_top=False
    height, width = buffer, buffer
    eff_net = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(height, width, nbands))

    eff_net.trainable = False
    
    return eff_net
    
def define_model(pretrained_model, n_fc):
    
    # pretrained_model can be efficientnet b0
    x = pretrained_model.output
    x = Flatten()(x)
    x = Dense(n_fc, activation="relu")(x)
    x = Dropout(0.5)(x)
    
    predictions = Dense(1, activation="sigmoid")(x)
    model = Model(input = pretrained_model.input, output = predictions)
    model.compile(optimizers.rmsprop(lr=1e-4, decay=1e-6),loss='binary_crossentropy',metrics=['accuracy'])

    return model

def get_model_type(img_type, buffer_len, merge_type, n_fc, batch_size):
    
    model_type = img_type + "_buf" + str(buffer_len) + "_" + merge_type + "_fc" + str(n_fc) + "_bs" + str(batch_size)
    
    return model_type

def train_model(path_model_base, X_tr, Y_tr, model, model_type, batch_size, n_epoch):
    
    # set the train and validation dataset
    X_train, X_val, Y_train, Y_val = train_test_split(X_tr, Y_tr, stratify=Y_tr, test_size=val_division)
    
    name_model = os.path.join(path_model_base, model_type+"_model.h5")
    
    #path_weights = os.path.join(path_model, 'weights{epoch:04d}.h5')
    #callback = ModelCheckpoint(path_weights, save_weights_only=True, period=10)
    # set early stopping for training
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
    
    print("Start Training", model_type)
    history = model.fit(X_train, Y_train,
                  batch_size=batch_size,
                  epochs=n_epoch,
                  callbacks=[callback],
                  validation_data=(X_val, Y_val), 
                  shuffle=True,
                  verbose=2)
    
    model.save(name_model)
    
    return history

def save_history(path_model_base, history, model_type):

    hist_df = pd.DataFrame(history.history) 

    # save to csv:
    hist_csv_file = os.path.join(path_model_base, "Model_"+model_type+"_history.csv") 
    
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)

#### Define paths

In [ ]:
# define path
path_array, path_model_base = define_path()

# set the type of images
img_type_list = ["NDVI", "PCA"]

# define the length of square buffer
buffer_list = [320, 480, 640, 800]

# dataset types
data_type_list = ["train", "test"]

# merge type
# partfull: merge partially functional and fully functional together
# partdest: merge partially functional and destroyed together
merge_type_list = ["partfull", "partdest"]

# define the hyperparameters for the model
n_fc_list = [16, 24, 32, 40]
batch_size_list = [16, 32, 64, 128, 256]

# fixed parameters
n_epoch = 100
nbands = 3
val_division = 0.2

#### Load train images and labels

In [ ]:
data_type = "train"

train_img_list, train_lab_partfull_list, train_lab_partdest_list = load_image_label_data(data_type, img_type_list, buffer_list, merge_type_list)

# train_img_dict, train_lab_partdest_dict, train_lab_partfull_dict include the arrays (images and labels) for training
train_img_dict, train_lab_partdest_dict, train_lab_partfull_dict = get_image_label_dict(train_img_list, img_type_list, buffer_list, merge_type_list, train_lab_partfull_list, train_lab_partdest_list)
train_lab_dict = [train_lab_partfull_dict, train_lab_partdest_dict]

#train_img_dict.keys(), train_lab_partdest_dict.keys(), train_lab_partfull_dict.keys()

img_type NDVI

Buffer 320
img shape (1993, 32, 32, 3)
partfull
lab shape (1993,)
partdest
lab shape (1993,)

Buffer 480
img shape (1956, 48, 48, 3)
partfull
lab shape (1956,)
partdest
lab shape (1956,)

Buffer 640
img shape (1925, 64, 64, 3)
partfull
lab shape (1925,)
partdest
lab shape (1925,)

Buffer 800
img shape (1888, 80, 80, 3)
partfull
lab shape (1888,)
partdest
lab shape (1888,)

img_type PCA

Buffer 320
img shape (1982, 32, 32, 3)
partfull
lab shape (1982,)
partdest
lab shape (1982,)

Buffer 480
img shape (1929, 48, 48, 3)
partfull
lab shape (1929,)
partdest
lab shape (1929,)

Buffer 640
img shape (1895, 64, 64, 3)
partfull
lab shape (1895,)
partdest
lab shape (1895,)

Buffer 800
img shape (1853, 80, 80, 3)
partfull
lab shape (1853,)
partdest
lab shape (1853,)



In [ ]:
# train the models
for i in range(len(img_type_list)):
    
    img_type = img_type_list[i]
    
    for j in range(len(buffer_list)):
        
        buffer_len = buffer_list[j]
        dict_key_img = img_type + "_Buf" + str(buffer_len)

        X_tr = train_img_dict[dict_key_img]
        pretrained_model = eff_net_b0(int(buffer_len/10))

        for k in range(len(merge_type_list)):
            
            merge_type = merge_type_list[k]
            dict_key_lab = img_type + "_Buf" + str(buffer_len) + "_" + merge_type
            Y_tr = train_lab_dict[k][dict_key_lab]
            
            for l in range(len(n_fc_list)):
                
                n_fc = n_fc_list[l]
                model = define_model(pretrained_model, n_fc)
                
                for m in range(len(batch_size_list)):
                    
                    batch_size = batch_size_list[m]
                    
                    model_type = get_model_type(img_type, buffer_len, merge_type, n_fc, batch_size)
                    print(model_type)
                    print("- "*40)
                    history = train_model(path_model_base, X_tr, Y_tr, model, model_type, batch_size, n_epoch)
                    
                    save_history(path_model_base, history, model_type)

C:\Users\b1077901\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:108: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


NDVI_buf320_partfull_fc16_bs16
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Start Training NDVI_buf320_partfull_fc16_bs16
Train on 1594 samples, validate on 399 samples
Epoch 1/100
 - 24s - loss: 0.8472 - accuracy: 0.5433 - val_loss: 0.6937 - val_accuracy: 0.5589
Epoch 2/100
 - 7s - loss: 0.7818 - accuracy: 0.5728 - val_loss: 0.7360 - val_accuracy: 0.5589
Epoch 3/100
 - 7s - loss: 0.7233 - accuracy: 0.6029 - val_loss: 0.7708 - val_accuracy: 0.5439
Epoch 4/100
 - 7s - loss: 0.6931 - accuracy: 0.6299 - val_loss: 0.6998 - val_accuracy: 0.5890
Epoch 5/100
 - 7s - loss: 0.6569 - accuracy: 0.6418 - val_loss: 0.6554 - val_accuracy: 0.6266
Epoch 6/100
 - 7s - loss: 0.6405 - accuracy: 0.6619 - val_loss: 0.6422 - val_accuracy: 0.6416
Epoch 7/100
 - 7s - loss: 0.6241 - accuracy: 0.6807 - val_loss: 0.6607 - val_accuracy: 0.6642
Epoch 8/100
 - 7s - loss: 0.5950 - accuracy: 0.6907 - val_loss: 0.6561 - val_accuracy: 0.6692
Epoch 9/100
 - 7s - loss: 0.6016 - accurac

 - 2s - loss: 0.0323 - accuracy: 0.9912 - val_loss: 0.1345 - val_accuracy: 0.9774
Epoch 14/100
 - 2s - loss: 0.0632 - accuracy: 0.9862 - val_loss: 0.1354 - val_accuracy: 0.9799
Epoch 15/100
 - 2s - loss: 0.0366 - accuracy: 0.9937 - val_loss: 0.1369 - val_accuracy: 0.9825
Epoch 16/100
 - 2s - loss: 0.0471 - accuracy: 0.9900 - val_loss: 0.1419 - val_accuracy: 0.9774
NDVI_buf320_partfull_fc16_bs128
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Start Training NDVI_buf320_partfull_fc16_bs128
Train on 1594 samples, validate on 399 samples
Epoch 1/100
 - 2s - loss: 0.0602 - accuracy: 0.9856 - val_loss: 0.0058 - val_accuracy: 0.9975
Epoch 2/100
 - 1s - loss: 0.0596 - accuracy: 0.9881 - val_loss: 0.0037 - val_accuracy: 1.0000
Epoch 3/100
 - 1s - loss: 0.0423 - accuracy: 0.9906 - val_loss: 0.0037 - val_accuracy: 1.0000
Epoch 4/100
 - 1s - loss: 0.0371 - accuracy: 0.9944 - val_loss: 0.0030 - val_accuracy: 1.0000
Epoch 5/100
 - 1s - loss: 0.0385 - accuracy: 0.989

C:\Users\b1077901\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:108: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


NDVI_buf320_partfull_fc24_bs16
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Start Training NDVI_buf320_partfull_fc24_bs16
Train on 1594 samples, validate on 399 samples
Epoch 1/100
 - 20s - loss: 0.4433 - accuracy: 0.7892 - val_loss: 0.1191 - val_accuracy: 0.9975
Epoch 2/100
 - 7s - loss: 0.2204 - accuracy: 0.9197 - val_loss: 0.0572 - val_accuracy: 1.0000
Epoch 3/100
 - 7s - loss: 0.1936 - accuracy: 0.9360 - val_loss: 0.0443 - val_accuracy: 0.9975
Epoch 4/100
 - 7s - loss: 0.1594 - accuracy: 0.9467 - val_loss: 0.0331 - val_accuracy: 0.9975
Epoch 5/100
 - 7s - loss: 0.1290 - accuracy: 0.9573 - val_loss: 0.0307 - val_accuracy: 0.9975
Epoch 6/100
 - 7s - loss: 0.1539 - accuracy: 0.9511 - val_loss: 0.0285 - val_accuracy: 0.9975
Epoch 7/100
 - 7s - loss: 0.1923 - accuracy: 0.9385 - val_loss: 0.0404 - val_accuracy: 0.9900
Epoch 8/100
 - 7s - loss: 0.1477 - accuracy: 0.9492 - val_loss: 0.0435 - val_accuracy: 0.9975
Epoch 9/100
 - 7s - loss: 0.1341 - accurac

...

PCA_buf800_partdest_fc24_bs16
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Start Training PCA_buf800_partdest_fc24_bs16
Train on 1482 samples, validate on 371 samples
Epoch 1/100
 - 26s - loss: 0.0860 - accuracy: 0.9764 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 2/100
 - 10s - loss: 0.0351 - accuracy: 0.9885 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 3/100
 - 10s - loss: 0.0370 - accuracy: 0.9919 - val_loss: 4.6444e-04 - val_accuracy: 1.0000
Epoch 4/100
 - 10s - loss: 0.0169 - accuracy: 0.9953 - val_loss: 1.4298e-04 - val_accuracy: 1.0000
Epoch 5/100
 - 10s - loss: 0.0232 - accuracy: 0.9953 - val_loss: 3.0108e-04 - val_accuracy: 1.0000
Epoch 6/100
 - 10s - loss: 0.0216 - accuracy: 0.9946 - val_loss: 3.7378e-04 - val_accuracy: 1.0000
Epoch 7/100
 - 10s - loss: 0.0345 - accuracy: 0.9939 - val_loss: 1.4541e-04 - val_accuracy: 1.0000
Epoch 8/100
 - 10s - loss: 0.0349 - accuracy: 0.9926 - val_loss: 7.5274e-04 - val_accuracy: 1.0000
Epoch 9/100
 

C:\Users\b1077901\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:108: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


PCA_buf800_partdest_fc32_bs16
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Start Training PCA_buf800_partdest_fc32_bs16
Train on 1482 samples, validate on 371 samples
Epoch 1/100
 - 25s - loss: 0.0592 - accuracy: 0.9798 - val_loss: 3.5268e-05 - val_accuracy: 1.0000
Epoch 2/100
 - 10s - loss: 0.0259 - accuracy: 0.9946 - val_loss: 1.5738e-04 - val_accuracy: 1.0000
Epoch 3/100
 - 10s - loss: 0.0231 - accuracy: 0.9966 - val_loss: 1.9945e-06 - val_accuracy: 1.0000
Epoch 4/100
 - 10s - loss: 0.0093 - accuracy: 0.9966 - val_loss: 2.8299e-04 - val_accuracy: 1.0000
Epoch 5/100
 - 10s - loss: 0.0194 - accuracy: 0.9980 - val_loss: 1.2059e-05 - val_accuracy: 1.0000
Epoch 6/100
 - 10s - loss: 0.0344 - accuracy: 0.9953 - val_loss: 3.1014e-05 - val_accuracy: 1.0000
Epoch 7/100
 - 10s - loss: 0.0117 - accuracy: 0.9960 - val_loss: 1.0582e-06 - val_accuracy: 1.0000
Epoch 8/100
 - 10s - loss: 0.0101 - accuracy: 0.9973 - val_loss: 1.6681e-04 - val_accuracy: 1.0000
Epoch

C:\Users\b1077901\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:108: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


PCA_buf800_partdest_fc40_bs16
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Start Training PCA_buf800_partdest_fc40_bs16
Train on 1482 samples, validate on 371 samples
Epoch 1/100
 - 25s - loss: 0.0239 - accuracy: 0.9899 - val_loss: 4.2366e-06 - val_accuracy: 1.0000
Epoch 2/100
 - 10s - loss: 0.0176 - accuracy: 0.9939 - val_loss: 3.2337e-06 - val_accuracy: 1.0000
Epoch 3/100
 - 10s - loss: 0.0099 - accuracy: 0.9939 - val_loss: 8.4902e-07 - val_accuracy: 1.0000
Epoch 4/100
 - 10s - loss: 0.0049 - accuracy: 0.9980 - val_loss: 1.9885e-07 - val_accuracy: 1.0000
Epoch 5/100
 - 10s - loss: 0.0028 - accuracy: 0.9993 - val_loss: 2.0718e-07 - val_accuracy: 1.0000
Epoch 6/100
 - 10s - loss: 0.0015 - accuracy: 0.9993 - val_loss: 7.0250e-08 - val_accuracy: 1.0000
Epoch 7/100
 - 10s - loss: 0.0084 - accuracy: 0.9973 - val_loss: 2.7367e-08 - val_accuracy: 1.0000
Epoch 8/100
 - 10s - loss: 0.0022 - accuracy: 0.9993 - val_loss: 1.5417e-08 - val_accuracy: 1.0000
Epoch

## Train models for augmented data

In [ ]:
def define_path(path):
    
    path_base = path
    path_model = os.path.join(path_base, "Models")
    path_npy_DA = os.path.join(path_base, "InputData", "DA")
    
    return path_base, path_model, path_npy_DA

def load_DA_img_lab_data(path_npy_DA, DA_type):
    
    new_img_name = "dataset_DA_Ori_"+DA_type+"_train_img.npy"
    new_lab_partfull_name = "dataset_DA_Ori_"+DA_type+"_train_partfull_lab.npy"
    new_lab_partdest_name = "dataset_DA_Ori_"+DA_type+"_train_partdest_lab.npy"

    new_img_path = os.path.join(path_npy_DA, new_img_name)
    new_lab_partfull_path = os.path.join(path_npy_DA, new_lab_partfull_name)
    new_lab_partdest_path = os.path.join(path_npy_DA, new_lab_partdest_name)

    DA_img = np.load(new_img_path)
    DA_lab_partfull = np.load(new_lab_partfull_path)
    DA_lab_partdest = np.load(new_lab_partdest_path)

    return DA_img, DA_lab_partfull, DA_lab_partdest

def get_model_type(img_type, buffer_len, merge_type, n_fc, batch_size, DA_type):
    
    model_type = img_type + "_buf" + str(buffer_len) + "_" + merge_type + "_fc" + str(n_fc) + "_bs" + str(batch_size)+"_"+DA_type
    
    return model_type

In [ ]:
# important variables
path_base, path_model, path_npy_DA = define_path(path)
img_type = "PCA"
merge_type = "partfull"
buffer_len = 640
n_fc = 24
batch_size = 256
nbands = 3
n_epoch = 100
val_division = 0.2

# variables of data augmentation
rotate_angle = 60
shift_pixel = 16
direction = "left"

In [ ]:
# define CNN model
pretrained_model = eff_net_b0(int(buffer_len/10))

model = define_model(pretrained_model, n_fc)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:108: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [ ]:
# test 1 - rotate
# load image and label data
DA_type = "rotate_"+str(rotate_angle)
model_type = get_model_type(img_type, buffer_len, merge_type, n_fc, batch_size, DA_type)
print(model_type)
DA_img, DA_lab_partfull, DA_lab_partdest = load_DA_img_lab_data(path_npy_DA, DA_type)
print(DA_img.shape, DA_lab_partfull.shape, DA_lab_partdest.shape)

X_tr = DA_img
Y_tr = DA_lab_partfull

history = train_model(path_model, X_tr, Y_tr, model, model_type, batch_size, n_epoch)
save_history(path_model, history, model_type)

# test 2 - translate
# load image and label data
DA_type = "translate_"+direction+"_"+str(shift_pixel)
model_type = get_model_type(img_type, buffer_len, merge_type, n_fc, batch_size, DA_type)
print(model_type)
DA_img, DA_lab_partfull, DA_lab_partdest = load_DA_img_lab_data(path_npy_DA, DA_type)
print(DA_img.shape, DA_lab_partfull.shape, DA_lab_partdest.shape)

X_tr = DA_img
Y_tr = DA_lab_partfull

history = train_model(path_model, X_tr, Y_tr, model, model_type, batch_size, n_epoch)
save_history(path_model, history, model_type)

# test 3 - rotate and translate
# load image and label data
DA_type = "rotate_"+str(rotate_angle)+"_translate_"+direction+"_"+str(shift_pixel)
model_type = get_model_type(img_type, buffer_len, merge_type, n_fc, batch_size, DA_type)
print(model_type)
DA_img, DA_lab_partfull, DA_lab_partdest = load_DA_img_lab_data(path_npy_DA, DA_type)
print(DA_img.shape, DA_lab_partfull.shape, DA_lab_partdest.shape)

X_tr = DA_img
Y_tr = DA_lab_partfull

history = train_model(path_model, X_tr, Y_tr, model, model_type, batch_size, n_epoch)
save_history(path_model, history, model_type)

PCA_buf640_partfull_fc24_bs256_rotate_60
(3790, 64, 64, 3) (3790,) (3790,)
Start Training PCA_buf640_partfull_fc24_bs256_rotate_60

Train on 3032 samples, validate on 758 samples
Epoch 1/100
 - 110s - loss: 0.8203 - accuracy: 0.5501 - val_loss: 4.7710 - val_accuracy: 0.5066
Epoch 2/100
 - 97s - loss: 0.6310 - accuracy: 0.6563 - val_loss: 4.3107 - val_accuracy: 0.4208
Epoch 3/100
 - 99s - loss: 0.5506 - accuracy: 0.7141 - val_loss: 3.6503 - val_accuracy: 0.4103
Epoch 4/100
 - 99s - loss: 0.5144 - accuracy: 0.7431 - val_loss: 2.3543 - val_accuracy: 0.4156
Epoch 5/100
 - 97s - loss: 0.4624 - accuracy: 0.7836 - val_loss: 1.5218 - val_accuracy: 0.4380
Epoch 6/100
 - 98s - loss: 0.4152 - accuracy: 0.8110 - val_loss: 1.1555 - val_accuracy: 0.5435
Epoch 7/100
 - 99s - loss: 0.3855 - accuracy: 0.8334 - val_loss: 0.8672 - val_accuracy: 0.5858
Epoch 8/100
 - 98s - loss: 0.3609 - accuracy: 0.8440 - val_loss: 0.8017 - val_accuracy: 0.6029
Epoch 9/100
 - 98s - loss: 0.3154 - accuracy: 0.8621 - val_l